# Student Career Pathway Recommender

This notebook implements a career recommender agent that suggests career paths based on student interests, hobbies, and academic strengths using LangChain and a free LLM from Hugging Face.

In [ ]:
#  Install required libraries
!pip install -q transformers gradio

#  Import Libraries
from transformers import pipeline
import gradio as gr
import json

#  Load a text-generation model
recommender = pipeline("text-generation", model="sshleifer/tiny-gpt2")

# Define career domain mapping
career_map = {
    "math": "STEM",
    "coding": "STEM",
    "science": "STEM",
    "biology": "STEM",
    "physics": "STEM",
    "chemistry": "STEM",
    "drawing": "Arts",
    "music": "Arts",
    "writing": "Arts",
    "painting": "Arts",
    "photography": "Arts",
    "soccer": "Sports",
    "cricket": "Sports",
    "basketball": "Sports",
    "fitness": "Sports",
    "coaching": "Sports"
}

#  Domain-based career options
career_options = {
    "STEM": [
        ("Software Engineer", "Uses your love for coding and math to build software."),
        ("Data Scientist", "Applies math and science to analyze and solve real-world problems."),
        ("Mechanical Engineer", "Designs machines using physics and problem-solving skills."),
        ("Research Scientist", "Explores scientific questions and experiments.")
    ],
    "Arts": [
        ("Graphic Designer", "Combines art and technology to create visuals."),
        ("Writer", "Uses creativity and language skills to communicate ideas."),
        ("Musician", "Expresses emotion and creativity through music."),
        ("Photographer", "Captures stories and scenes visually.")
    ],
    "Sports": [
        ("Athletic Coach", "Trains athletes and helps improve performance."),
        ("Sports Analyst", "Uses data to evaluate sports performance."),
        ("Fitness Trainer", "Helps others stay fit and active."),
        ("Professional Athlete", "Pursues sports competitively.")
    ],
    "General": [
        ("Teacher", "Helps educate and guide students."),
        ("Entrepreneur", "Builds new businesses based on passion areas."),
        ("Journalist", "Researches and reports on current events.")
    ]
}

#  Extract matching domains

def get_domains(user_input):
    domains = set()
    for keyword, domain in career_map.items():
        if keyword in user_input.lower():
            domains.add(domain)
    return list(domains) if domains else ["General"]

# Career Recommender Logic

def recommend_all_careers(user_input):
    if len(user_input.strip().split()) < 4:
        return json.dumps({
            "fallback": True,
            "message": "Please provide more detail about your interests, hobbies, or strengths."
        }, indent=2)

    matched_domains = get_domains(user_input)
    all_recommendations = []

    for domain in matched_domains:
        for career, explanation in career_options.get(domain, []):
            all_recommendations.append({
                "career_path": career,
                "explanation": explanation
            })

    return json.dumps({
        "matched_domains": matched_domains,
        "recommendations": all_recommendations
    }, indent=2)

#  Gradio GUI
with gr.Blocks() as demo:
    gr.Markdown("""
    # 🎓 Student Career Pathway Recommender
    Enter your interests, hobbies, and academic strengths below to receive multiple career suggestions with explanations.
    """)

    user_input = gr.Textbox(label="Enter your interests and strengths")
    output = gr.Code(label="Career Recommendations (JSON)")
    btn = gr.Button("Get Recommendations")

    btn.click(fn=recommend_all_careers, inputs=user_input, outputs=output)

# Launch
demo.launch(share=True)


# New Section